In [1]:
# 라이브러리
import os
import glob
import zlib
import struct
import olefile
import re

In [3]:
import os
import re
import hashlib
import unicodedata

def safe_filename(original_name: str, suffix: str = "_parsed.txt", max_bytes: int = 180) -> str:
    """
    OS 파일명 길이(바이트) 제한을 피하기 위해:
    - 원본 이름 정규화
    - 위험한 문자 제거
    - UTF-8 바이트 기준으로 자르기
    - 짧은 해시를 붙여 충돌 방지
    """
    name = unicodedata.normalize("NFC", original_name)

    # 확장자 제거 (혹시 .hwp가 여러 번 들어가도 안전하게)
    base = re.sub(r"\.hwp$", "", name, flags=re.IGNORECASE)

    # 파일명에 들어가면 곤란한 문자 제거/치환
    base = re.sub(r"[\/\\\:\*\?\"\<\>\|\n\r\t]+", " ", base)  # 윈도우 금지문자 포함
    base = re.sub(r"\s+", " ", base).strip()

    # 충돌 방지용 짧은 해시
    h = hashlib.sha1(name.encode("utf-8")).hexdigest()[:10]

    # suffix 고려해서 base를 바이트 기준으로 자르기
    # 최종 파일명: "{base}__{hash}{suffix}"
    tail = f"__{h}{suffix}"
    budget = max_bytes - len(tail.encode("utf-8"))
    if budget < 20:
        budget = 20

    b = base.encode("utf-8")
    if len(b) > budget:
        base = b[:budget].decode("utf-8", errors="ignore").rstrip()

    return f"{base}__{h}{suffix}"


In [4]:
def parse_hwp_all(file_path: str) -> str:
    """HWP 파일에서 전체 텍스트를 추출합니다. (BodyText 파싱)"""
    if not os.path.exists(file_path):
        print(f"'{file_path}' 파일을 찾을 수 없습니다.")
        return ""
    try:
        f = olefile.OleFileIO(file_path)
        
        # 압축 여부 확인
        header = f.openstream("FileHeader").read()
        is_compressed = header[36] & 1
        
        texts = []
        
        # 모든 BodyText 섹션 처리
        for entry in f.listdir():
            if entry[0] == "BodyText":
                data = f.openstream(entry).read()
                
                # 압축 해제
                if is_compressed:
                    try:
                        data = zlib.decompress(data, -15)
                    except:
                        pass
                
                # 레코드 파싱
                i = 0
                while i < len(data):
                    if i + 4 > len(data):
                        break
                    
                    rec_header = struct.unpack_from('<I', data, i)[0]
                    rec_type = rec_header & 0x3FF
                    rec_len = (rec_header >> 20) & 0xFFF
                    
                    # 확장 길이 처리
                    if rec_len == 0xFFF:
                        if i + 8 > len(data):
                            break
                        rec_len = struct.unpack_from('<I', data, i + 4)[0]
                        i += 8
                    else:
                        i += 4
                    
                    if i + rec_len > len(data):
                        break
                    
                    # HWPTAG_PARA_TEXT (67)
                    if rec_type == 67 and rec_len > 0:
                        text_data = data[i:i + rec_len]
                        try:
                            text = text_data.decode('utf-16le', errors='ignore')
                            # HWP 제어문자 제거 (0x00~0x1F 범위의 특수 제어코드)
                            cleaned = []
                            for char in text:
                                code = ord(char)
                                if code >= 32 or char in '\n\r\t':
                                    cleaned.append(char)
                                elif code in [13, 10]:  # CR, LF
                                    cleaned.append('\n')
                            text = ''.join(cleaned).strip()
                            if text:
                                texts.append(text)
                        except:
                            pass
                    
                    i += rec_len
        
        f.close()
        
        if texts:
            result = '\n'.join(texts)
            
            # 1. 깨진 문자 제거 (한글, 영문, 숫자, 공백, 로마숫자, 기본 문장부호 유지)
            result = re.sub(r'[^\uAC00-\uD7A3\u2160-\u217Fa-zA-Z0-9\s.,!?():\-\[\]<>~·%/@\'"_=+○●■□▶◆※]', '', result)
            
            # 2. 연속 공백/줄바꿈 정리
            result = re.sub(r'\n{3,}', '\n\n', result)
            result = re.sub(r' {2,}', ' ', result)
            
            print(f"'{file_path}' 파일 파싱 성공!")
            return result.strip()
        else:
            print(f"'{file_path}' 텍스트 추출 실패")
            return ""
            
    except Exception as e:
        print(f"'{file_path}' 파일 파싱 중 오류 발생: {e}")
        return ""

In [5]:
# data 폴더의 모든 HWP 파일 파싱
hwp_files = glob.glob("data/*.hwp")
print(f"발견된 HWP 파일: {len(hwp_files)}개\n")

OUT_DIR = "data_parsed"
os.makedirs(OUT_DIR, exist_ok=True)

parsed_docs = {}
mapping = []  # 원본파일명 ↔ 저장파일명 기록 (RAG 메타데이터용)

for file_path in hwp_files:
    file_name = os.path.basename(file_path)
    print(f"--- {file_name} ---")
    text = parse_hwp_all(file_path)

    if text:
        parsed_docs[file_name] = text
        print(f"[추출된 텍스트 미리보기]")
        print(text[:500])
        print("...\n")

        # ✅ txt 파일로 저장 (파일명 짧게 + 안전하게)
        safe_name = safe_filename(file_name, suffix="_parsed.txt", max_bytes=180)
        output_file = os.path.join(OUT_DIR, safe_name)

        if os.path.exists(output_file):
            os.remove(output_file)
            print(f"기존 파일 삭제: {output_file}")

        with open(output_file, "w", encoding="utf-8") as f:
            f.write(text)

        mapping.append({
            "original_filename": file_name,
            "hwp_path": file_path,
            "parsed_txt_path": output_file,
            "chars": len(text)
        })

        print(f"저장 완료: {output_file} ({len(text)}자)\n")
    else:
        print("텍스트 추출 실패\n")

# ✅ 매핑 저장(강추): 나중에 청킹/임베딩할 때 source로 씀
import json
with open(os.path.join(OUT_DIR, "parsed_mapping.json"), "w", encoding="utf-8") as f:
    json.dump(mapping, f, ensure_ascii=False, indent=2)

print(f"총 {len(parsed_docs)}/{len(hwp_files)}개 파일 파싱 완료")
print(f"✅ 매핑 저장: {os.path.join(OUT_DIR, 'parsed_mapping.json')}")


발견된 HWP 파일: 96개

--- 사단법인아시아물위원회사무국_우즈벡-키르기즈스탄 기후변화대응 스.hwp ---
'data/사단법인아시아물위원회사무국_우즈벡-키르기즈스탄 기후변화대응 스.hwp' 파일 파싱 성공!
[추출된 텍스트 미리보기]
제 안 요 청 서

우즈벡-키르기즈스탄 기후변화대응 스마트 관개시스템 구축사업

2024. 10.
아시아 물 위원회

목 차
Ⅰ. 제안 요청내용		 01
1. 용역개요		 02
2. 기본방향		 03
3. 세부 과업지침		 05
Ⅲ. 입찰 안내사항		 13
1. 입찰참가자격		 13
2. 입찰 및 낙찰자 선정		 14
3. 입찰참가 등록 및 입찰서 제출 		 15
4. 입찰시 유의사항 		 16
5. 청렴계약이행 준수 		 17
6. 제안서 평가 및 용역기관 선정 		 17
7. 기타사항 		 22
8. 최종낙찰자 유의사항 		 22
Ⅳ. 제안서 작성 안내사항		 25
1. 제안 안내사항		25
2. 제안서 작성방법		27

Ⅴ. 각종기준 등		 33
1. 정량평가 기준		 33
2. 정성평가 기준		 40
3. 용역비 산정, 집행 및 정산기준		 42
4. 전문가파견 가이드라인		 53
Ⅵ. 서식		 61
[별지 제1호 서식] 정성, 정량제안서(표지)		62
[별지 제2호 서식] 
...

저장 완료: data_parsed/사단법인아시아물위원회사무국_우즈벡-키르기즈스탄 기후변화대응 스__5b75765779_parsed.txt (46350자)

--- 축산물품질평가원_축산물이력관리시스템 개선(정보화 사업).hwp ---
'data/축산물품질평가원_축산물이력관리시스템 개선(정보화 사업).hwp' 파일 파싱 성공!
[추출된 텍스트 미리보기]


'data/한국생산기술연구원_2세대 전자조달시스템  기반구축사업.hwp' 파일 파싱 성공!
[추출된 텍스트 미리보기]
2세대 전자조달시스템
기반구축사업
2024. 4.

디지털행정추진실
한국생산기술연구원

담당
사양 및 과업
디지털행정추진실
배소연
TEL:
041-589-8092
FAX :
041-589-8090
입찰 및 계약
구매자산실
이승옥
TEL:
041-589-8547
FAX:
041-589-8540

목 차
Ⅰ. 과업안내		 3
1. 과업개요		 3
2. 추진목적		 3
3. 추진전략		 3
4. 추진체계		 3
Ⅱ. 과업내용		 4
1. 과업내용		 4
2. 상세 요구사항		 8
3. 추진일정		 49
4. 산출물		 49
Ⅲ. 과업수행 일반사항		 50
Ⅳ. 제안 일반사항		 60
1. 입찰참가서류 및 제안서 제출		 60
2. 지원자격 및 과업내용변경 사항		 60
3. 제안서 작성 요령		 60
4. 제안서 목차(안)		 63
Ⅴ. 제안서 평가		 66
1. 협상절차 및 낙찰자 결정 방식		 66
2. 평가표 및 배점		 67
<첨부> 보안 관련 안내		 72
<별지> 보안 관련 양
...

저장 완료: data_parsed/한국생산기술연구원_2세대 전자조달시스템 기반구축사업__c0df334881_parsed.txt (68815자)

--- (사)벤처기업협회_2024년 벤처확인종합관리시스템 기능 고도화 용역사업 .hwp ---
'data/(사)벤처기업협회_2024년 벤처확인종합관리시스템 기능 고도화 용역사업 .hwp' 파일 파싱 성공!
[추출된 텍스트 미리보기]
2024년 벤처확인종합관리시스템 기능 고도화 용역사업
(복수의결권주식, 스톡옵션, 성과조건부주식) -
제안요청서
2024. 03.

목 차
1. 추진개요		 3
2. 추

In [6]:
def paragraph_chunking(
    text: str,
    min_chars: int = 200,
    max_chars: int = 800,
    overlap: int = 100
):
    """
    문단 기반 청킹
    - 빈 줄 기준으로 문단 분리
    - 너무 짧은 문단은 합침
    - 너무 긴 문단은 재분할
    """
    paragraphs = [p.strip() for p in text.split("\n\n") if p.strip()]
    chunks = []

    buffer = ""
    for p in paragraphs:
        if len(buffer) + len(p) <= max_chars:
            buffer += ("\n\n" + p) if buffer else p
        else:
            if len(buffer) >= min_chars:
                chunks.append(buffer)
                buffer = p
            else:
                buffer += ("\n\n" + p)

    if buffer:
        chunks.append(buffer)

    # 너무 긴 chunk 재분할 + overlap
    final_chunks = []
    for c in chunks:
        if len(c) <= max_chars:
            final_chunks.append(c)
        else:
            start = 0
            while start < len(c):
                end = start + max_chars
                final_chunks.append(c[start:end])
                start = end - overlap

    return final_chunks


In [7]:
chunks = paragraph_chunking(text)

print(f"총 chunk 수: {len(chunks)}")
print(chunks[0][:500])


총 chunk 수: 83
제 안 요 청 서

사 업 명
강릉어선안전국 상황관제시스템 구축사업
발주기관
수산업협동조합중앙회 어선안전조업본부
2025. 2.

담당
소 속
성 명
전 화
팩 스
어선안전조업본부
어선ICT지원팀
정현우
02-2240-2317
02-2240-3028
전요셉
02-2240-2382

목 차
Ⅰ. 사업의 안내
1. 사업설명 		1
2. 사업개요 		1
3. 사업범위		2
4. 추진일정		2
5. 기대효과		2
Ⅱ. 제안요청 내용
1. 목표시스템 구성도		3
2. 주요사업내용		3
3. 요구사항목록		5
4. 요구사항상세		6
Ⅲ. 사업자 선정 및 계약 방법
1. 입찰참가 자격		28
2. 사업자 선정 방법		29
3. 입찰참가(제안서) 등록 및 제출서류		34
4. 입찰 참가시 조건사항		35
5. 제안서 작성요령		36
6. 제안서의 효력		36
7. 제안평가회 및 평가방법		37
※ 별지(별첨)서식
별지서식 1~13		39
별첨 1~3		58
※ 붙임서식
안전보건관리 가이드 붙임 1


In [ ]:
chunk_docs = []
for i, c in enumerate(chunks):
    chunk_docs.append({
        "chunk_id": i,
        "text": c,
        "metadata": {
            "source": file_name,
            "type": "paragraph"
        }
    })


In [13]:
from langchain_core.documents import Document



docs = [
    Document(page_content=d["text"], metadata=d["metadata"] | {"chunk_id": d["chunk_id"]})
    for d in chunk_docs
]

print("docs:", len(docs))
print(docs[0].metadata)
print(docs[0].page_content[:300])


docs: 83
{'source': '수협중앙회_강릉어선안전조업국 상황관제시스템 구축.hwp', 'type': 'paragraph', 'chunk_id': 0}
제 안 요 청 서

사 업 명
강릉어선안전국 상황관제시스템 구축사업
발주기관
수산업협동조합중앙회 어선안전조업본부
2025. 2.

담당
소 속
성 명
전 화
팩 스
어선안전조업본부
어선ICT지원팀
정현우
02-2240-2317
02-2240-3028
전요셉
02-2240-2382

목 차
Ⅰ. 사업의 안내
1. 사업설명 		1
2. 사업개요 		1
3. 사업범위		2
4. 추진일정		2
5. 기대효과		2
Ⅱ. 제안요청 내용
1. 목표시스템 구성도		3
2. 주요사업내용		3
3. 요구사항목록		5
4. 요구사항상세		6
Ⅲ. 사업


In [ ]:
from langchain_community.vectorstores import FAISS
from langchain_community.embeddings import HuggingFaceEmbeddings

# ✅ 로컬 임베딩(가벼움 + 성능 괜찮음)
embedding = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

FAISS_DIR = "faiss_index_hwp_v1"

# ✅ 벡터DB 생성
vectorstore = FAISS.from_documents(docs, embedding)

# ✅ 저장
vectorstore.save_local(FAISS_DIR)
print("✅ saved:", FAISS_DIR)


/home/sangjin1005/psj_folder/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Loading weights: 100%|██████████| 103/103 [00:00<00:00, 467.42it/s, Materializing param=pooler.dense.weight]                             
BertModel LOAD REPORT from: sentence-transformers/all-MiniLM-L6-v2
Key                     | Status     |  | 
------------------------+------------+--+-
embeddings.position_ids | UNEXPECTED |  | 

Notes:
- UNEXPECTED	:can be ignored when loading from different task/architecture; not ok if you expect identical arch.


In [ ]:
retriever = vectorstore.as_retriever(search_kwargs={"k": 5})

q = "DNS lookup에 의존한 보안결정이 왜 취약점이야?"
hits = retriever.invoke(q)

print("검색 결과:", len(hits))
for i, d in enumerate(hits):
    print("-"*80)
    print(f"[{i}] source={d.metadata.get('source')} chunk_id={d.metadata.get('chunk_id')}")
    print(d.page_content[:400])
